In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, to_date

# Создаем сессию Spark
spark = SparkSession.builder \
    .appName("Log Analysis") \
    .getOrCreate()

25/08/08 12:27:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [25]:
# Загружаем данные логов в DataFrame
logs_df = spark.read.csv("web_server_logs.csv", header=True, inferSchema=True)

In [14]:
# 1. Сгруппировать данные по IP и посчитать количество запросов для каждого IP
active_ips = logs_df.groupBy("ip").agg(count("*").alias("request_count")) \
    .orderBy(col("request_count").desc()) \
    .limit(10)

print("Top 10 active IP addresses:")
active_ips.show()

Top 10 active IP addresses:
+--------------+-------------+
|            ip|request_count|
+--------------+-------------+
| 215.176.48.26|            2|
|153.115.56.165|            1|
|174.65.158.114|            1|
| 142.46.150.38|            1|
|  30.123.67.99|            1|
|116.45.191.160|            1|
| 83.147.47.217|            1|
|24.220.176.221|            1|
| 219.67.80.249|            1|
|56.177.216.254|            1|
+--------------+-------------+



In [16]:
# 2. Сгруппировать данные по HTTP-методу и посчитать количество запросов для каждого метода
http_methods = logs_df.groupBy("method").agg(count("*").alias("request_count"))

print("Request count by HTTP method:")
http_methods.show()

Request count by HTTP method:
+------+-------------+
|method|request_count|
+------+-------------+
|  POST|        25245|
|DELETE|        24871|
|   PUT|        25029|
|   GET|        24855|
+------+-------------+



In [17]:
# 3. Профильтровать и посчитать количество запросов с кодом ответа 404
not_found_count = logs_df.filter(col("response_code") == 404).count()

print(f"Number of 404 response codes: {not_found_count}")

Number of 404 response codes: 25203


In [29]:
# 4. Сгруппировать данные по дате и просуммировать размер ответов, сортируя по дате
response_size_by_date = logs_df.groupBy(to_date(col("timestamp")).alias("date")) \
    .agg(sum("response_size").alias("total_response_size")) \
    .orderBy("date")

print("Total response size by day:")
response_size_by_date.show()

Total response size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2025-01-01|            2378848|
|2025-01-02|            2261159|
|2025-01-03|            2249441|
|2025-01-04|            2116138|
|2025-01-05|            2287015|
|2025-01-06|            2295584|
|2025-01-07|            2243026|
|2025-01-08|            2044661|
|2025-01-09|            2484495|
|2025-01-10|            2257060|
|2025-01-11|            2342658|
|2025-01-12|            2220594|
|2025-01-13|            2291610|
|2025-01-14|            2311337|
|2025-01-15|            2445778|
|2025-01-16|            2152530|
|2025-01-17|            2216470|
|2025-01-18|            2184526|
|2025-01-19|            2574248|
|2025-01-20|            2247567|
+----------+-------------------+
only showing top 20 rows



In [30]:
# Остановка сессии Spark
spark.stop()